In [11]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import itertools
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
df = pd.read_csv("churn_data.csv")
data = df.copy()

In [3]:
categorical = ['Geography', 'Gender', 'HasCrCard', 'IsActiveMember']
to_drop = ['RowNumber', 'CustomerId', 'Surname', 'Exited']
continuous = ['CreditScore', 'Age', 'Balance', 'Tenure', 'NumOfProducts', 'EstimatedSalary']

In [4]:
def prepare_data(X):
    
    y = X['Exited'] # таргет
    
    # определяем вещественный и категориальные фичи, а так же те, котороые мы будем дропать
    categorical = ['Geography', 'Gender', 'HasCrCard', 'IsActiveMember']
    to_drop = ['RowNumber', 'CustomerId', 'Surname', 'Exited']
    continuous = ['CreditScore', 'Age', 'Balance', 'Tenure', 'NumOfProducts', 'EstimatedSalary']
    
    X.drop(columns=to_drop, inplace=True)
    
    # OneHotEncoding для категорий:
    for col in categorical:
        to_add = pd.get_dummies(X[col], prefix=col)
        for column in to_add.columns:
            X[column] = to_add[column]
        X.drop(columns=col, inplace=True)
    
    return X, y

In [5]:
X, y = prepare_data(data)

Разбиваем на тренировочный и валидационный датасеты

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

Применяем стандартизацию

In [7]:
scaler = StandardScaler()
X_train[continuous] = scaler.fit_transform(X_train[continuous])
X_test[continuous] = scaler.transform(X_test[continuous])

Тестируем разные модели:

In [51]:
lr = LogisticRegression()
cat = CatBoostClassifier(silent=True)
rf = RandomForestClassifier()
ada = AdaBoostClassifier()

models = [lr, cat, rf, ada] # создаем список из моделей

results = pd.DataFrame(columns=['Model', 'TP', 'FP', 'Precision', 'Recall', 'Fscore', 'Model_profit'])

for model in models:
    model.fit(X_train, y_train)
    preds = model.predict_proba(X_test)[:, 1]
    
    precision, recall, thresholds = precision_recall_curve(y_test, preds)

    beta = 1 # создаем переменную бета, которую используем для подсчета fscore. 
            
    fscore = ((1 + beta**2) * precision * recall) / (beta**2 * precision + recall)
    ix = np.argmax(fscore)
    
    tn, fp, fn, tp = confusion_matrix(y_test, preds>thresholds[ix]).ravel()
    
    # считаем прибыль от модели. 2 доллара за TP, и вычитаем расходы в 1 доллар на все Positive predictions.   
    profit_from_model = 2*tp - (fp + tp) 
    
    results = results.append({'Model': str(model).split("(")[0], 
                              'TP': tp,
                              'FP': fp,
                              'Precision': precision[ix], 
                              'Recall': recall[ix],
                              'Fscore': fscore[ix],
                              'Model_profit': profit_from_model}, ignore_index=True)

results

,Model,TP,FP,Precision,Recall,Fscore,Model_profit
0,LogisticRegression,319,437,0.422721,0.628684,0.505529,-118
1,<catboost.core.CatBoostClassifier object at 0x...,310,150,0.674620,0.611002,0.641237,160
2,RandomForestClassifier,288,108,0.723716,0.581532,0.644880,180
3,AdaBoostClassifier,279,110,0.717949,0.550098,0.622914,169


Логистическая регрессия убыточна, остальные модели держатся примерно на одном уровне, но возьмем RandomForest для дальнейшего изучения. Заметно, что у RF самый большой профит, а так же самый большой Precision. Поэтому будем обращать особое внимание на Precision, так как эта метрика является ключевой для решения этой задачи. Однако, это не значит, что нам нужно обучать модель до состояния Precision = 1, нам нужно выбрать такой F-score, в котором Precision будет иметь чуть больший вес, а значит далее при расчетах fscore возьмем beta=0.5 и будем оринетироваться не на F1 score, а на F0.5 score.

In [49]:
tuned_results = pd.DataFrame(columns=['n_estimators', 
                                      'min_samples_leaf',
                                      'max_depth',
                                      'TP', 
                                      'FP', 
                                      'Precision', 
                                      'Recall', 
                                      'Fscore', 
                                      'Model_profit'])

# поищем наилучшие параметры для случайного леса:
n_estimators = [300, 400, 500, 600, 700]
min_samples_leaf = [1, 2, 3, 4, 5]
max_depth = [6, 7, 8, 9]

for estimator in n_estimators:
    for leaf in min_samples_leaf:
        for depth in max_depth:
            rf_tuned = RandomForestClassifier(max_depth=depth, min_samples_leaf=leaf, n_estimators=estimator)

            rf_tuned.fit(X_train, y_train)
            preds = rf_tuned.predict_proba(X_test)[:, 1]

            precision, recall, thresholds = precision_recall_curve(y_test, preds)
            
            beta = 0.5 # устанавливаем 0.5, таким образом увеличивая важность precision.
            
            fscore = ( (1 + beta**2) * precision * recall) / (beta**2 * precision + recall)
            ix = np.argmax(fscore)

            tn, fp, fn, tp = confusion_matrix(y_test, preds>thresholds[ix]).ravel()

            profit_from_model = 2*tp - (fp + tp)

            tuned_results = tuned_results.append({'n_estimators': estimator, 
                                                  'min_samples_leaf': leaf,
                                                  'max_depth': depth,
                                                  'TP': tp,
                                                  'FP': fp,
                                                  'Precision': precision[ix], 
                                                  'Recall': recall[ix],
                                                  'Fscore': fscore[ix],
                                                  'Model_profit': profit_from_model}, ignore_index=True)

In [50]:
# сортируем по профиту
tuned_results.sort_values(by='Model_profit', ascending=False).head(1)

,n_estimators,min_samples_leaf,max_depth,TP,FP,Precision,Recall,Fscore,Model_profit
46,500.0,2.0,8.0,246.0,51.0,0.828859,0.485265,0.726044,195.0


Наилучшая модель с параметрами выдает профит в 195. Видим, что Precision на высоком уровне, Recall гораздо ниже, но для того, чтобы решить именно задачу максимального профита, мы максимизируем Precision. 